In [2]:
import numpy as np
import pandas as pd
import networkx as nx

from polygraphs.analysis import Processor

/Users/prudhvivuda/Documents/polygraphs/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
class BeliefProcessor(Processor):
    def __init__(self, path):
        super().__init__(path)
        # Add the columns when processor is initialized
        self.add(self.mean(0.5, 0.75, 0.9, 0.99), self.quantile(0.5, 0.75, 0.9, 0.99, quantiles=(0.25,)))
        

    def mean(self, *thresholds):
        """Use the beliefs dataframe to find when mean beliefs first met a given threshold and count threshold crossings."""
        def get_mean_with_recrossings(iterations, threshold=0.5):
            # Average belief of all nodes at each iteration
            mean_iteration = iterations.groupby("iteration").mean()

            # Track crossing events
            above_threshold = mean_iteration["beliefs"] > threshold
            crossings = ((above_threshold & ~above_threshold.shift(fill_value=False))|
                         (~above_threshold & above_threshold.shift(fill_value=False))).sum()
            
            # Detect first crossing
            first_crossing = mean_iteration[above_threshold].index[0] if above_threshold.any() else None
            
            return first_crossing, crossings
        
        # Loop through list of arguments
        for threshold in thresholds:
            threshold = float(threshold)
            if 0 <= threshold <= 1:
                # Call the function for each simulation belief
                first_crossing_list = []
                crossing_count_list = []
                for belief in self.beliefs:
                    first_crossing, crossing_count = get_mean_with_recrossings(belief, threshold)
                    first_crossing_list.append(first_crossing)
                    crossing_count_list.append(crossing_count)

                # Add columns to the dataframe
                column = "mean_" + str(threshold).replace(".", "_")
                crossing_column = column + "_crossings"
                self.dataframe[column] = first_crossing_list
                self.dataframe[crossing_column] = crossing_count_list

    def quantile(self, *thresholds, quantiles=(0.25,)):
        def get_quantile(iterations, threshold, quantile_level):
            # Find the specified quantile of all nodes at each iteration
            quantile_iteration = iterations.groupby("iteration").quantile(quantile_level)
            
            above_threshold = quantile_iteration['beliefs'] > threshold
            count_crossings = ((above_threshold & ~above_threshold.shift(fill_value=False))|
                               (~above_threshold & above_threshold.shift(fill_value=False))).sum()
            
            # Filter out the iterations that do not meet threshold
            iterations_above_threshold = quantile_iteration[quantile_iteration['beliefs'] > threshold]
            
            # Check that we found a threshold
            if len(iterations_above_threshold) > 0:
                # Return the first remaining iteration (the index) and count of crossings
                return iterations_above_threshold.index[0], count_crossings
            else:
                return None, count_crossings

        # Loop through thresholds and quantiles
        for threshold in thresholds:
            threshold = float(threshold)
            if 0 <= threshold <= 1:
                for quantile_level in quantiles:
                    if 0 <= quantile_level <= 1:
                        # Call get_quantile function on each simulation beliefs
                        quantile_list = []
                        crossings_list = []
                        for belief in self.beliefs:
                            first_crossing, crossing_count = get_quantile(belief, threshold, quantile_level)
                            quantile_list.append(first_crossing)
                            crossings_list.append(crossing_count)
                        
                        # Add columns to the dataframe
                        column = f"quantile_{str(threshold).replace('.', '_')}_q{int(quantile_level * 100)}"
                        crossing_column = column + "_crossings"
                        self.dataframe[column] = quantile_list
                        self.dataframe[crossing_column] = crossings_list



In [4]:
x = BeliefProcessor("~/polygraphs-cache/results/")
x.get()

,bin_file_path,hd5_file_path,config_json_path,trials,network_kind,op,epsilon,uid,mean_0_5,mean_0_5_crossings,...,mean_0_99,mean_0_99_crossings,quantile_0_5_q25,quantile_0_5_q25_crossings,quantile_0_75_q25,quantile_0_75_q25_crossings,quantile_0_9_q25,quantile_0_9_q25_crossings,quantile_0_99_q25,quantile_0_99_q25_crossings
0,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,5a76ccf32d5649819dc51accaacad175,200,1,...,None,0,3200.0,1,5400.0,1,9800.0,5,15300.0,1
1,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,5a76ccf32d5649819dc51accaacad175,0,1,...,None,0,2100.0,5,5400.0,3,7900.0,3,13700.0,7
2,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,5a76ccf32d5649819dc51accaacad175,100,1,...,None,0,2600.0,5,5600.0,3,7500.0,9,13600.0,3
3,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,5a76ccf32d5649819dc51accaacad175,300,1,...,None,0,3900.0,4,NaN,0,NaN,0,NaN,0
4,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,80c5d95fa1ec46e2b8ccffa2f122a14c,200,1,...,None,0,2000.0,9,4700.0,3,7400.0,7,15700.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,0cda501d6f1e40fc88ce3856372d3dc3,0,1,...,None,0,NaN,0,NaN,0,NaN,0,NaN,0
71,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,298e3861a5f54d038770534ee097ccc8,0,1,...,None,0,2000.0,3,4300.0,1,7800.0,9,13600.0,1
72,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,298e3861a5f54d038770534ee097ccc8,1,1,...,None,0,3200.0,3,5800.0,5,9700.0,5,17900.0,1
73,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,/Users/prudhvivuda/polygraphs-cache/results/20...,64.0,snap,UnreliableNetworkModifiedAlignedBinomialOp,0.001,298e3861a5f54d038770534ee097ccc8,0,1,...,None,0,3000.0,5,5600.0,7,10200.0,7,14900.0,1


In [5]:
x.get().shape

(75, 24)

In [6]:
x.get().count()

bin_file_path                  75
hd5_file_path                  75
config_json_path               75
trials                         75
network_kind                   75
op                             75
epsilon                        75
uid                            75
mean_0_5                       75
mean_0_5_crossings             75
mean_0_75                      68
mean_0_75_crossings            75
mean_0_9                       58
mean_0_9_crossings             75
mean_0_99                       0
mean_0_99_crossings            75
quantile_0_5_q25               70
quantile_0_5_q25_crossings     75
quantile_0_75_q25              61
quantile_0_75_q25_crossings    75
quantile_0_9_q25               58
quantile_0_9_q25_crossings     75
quantile_0_99_q25              57
quantile_0_99_q25_crossings    75
dtype: int64